In [ ]:
# check getloader return type supposed to be lr_image, hr_image
from model import SRResnetConfig, LitSRResnet
from dataset import get_dataloader

train_dataset,test_dataset =get_dataloader(batch_size=1, num_workers=1)


In [ ]:
for x, y in train_dataset:
    print(x.shape, y.shape)
    break

In [ ]:
# check model architecuture, whethere there are incopatible channel sizes
import torch
from sr.resnet.model import *
config = SRResnetConfig(
    hidden_channel=3,
    device='cuda',
    lr=1e-3,
    is_training=True
)
model = LitSRResnet(config)

y = model(torch.rand(1, 3, 12, 12))
y.shape

In [ ]:
import pytorch_lightning as L
from pytorch_lightning.callbacks import ModelCheckpoint


checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',   # Metric to monitor
    filename='model',  # Filename for the best checkpoint
    save_top_k=3,          # Save only the best model
    mode='min'             # Save the model with minimum validation loss
)

# train using lightning
trainer = L.Trainer(max_epochs=2, callbacks=[checkpoint_callback])
trainer.fit(model, train_dataloaders=train_dataset, val_dataloaders=val_dataset)
